In [ ]:
cd ..

In [ ]:
!source venv/bin/activate

In [ ]:
ls

In [ ]:
from datasets import load_dataset
from tqdm.notebook import tqdm
from langchain_text_splitters import RecursiveCharacterTextSplitter
import pandas as pd
import numpy as np

from research.functions import get_natural_questions_sample, get_ms_marco_sample
from rag_techniques.simple_rag_giga import SimpleRag

# Разбиение на чанки

In [ ]:
from database.vbase import QdrantBase
from langchain_experimental.text_splitter import SemanticChunker
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

In [ ]:
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs={'device': 'cuda'}
)

In [ ]:
semantic_chunker = SemanticChunker(
    embeddings=embeddings,
    breakpoint_threshold_type='percentile',
)

In [ ]:
dataset = {
    'question': [],
    'answer': [],
}
texts = []

null_count = 0
max_null = 100

for data in tqdm(natural_questions):
    if len(texts) == 500:
        break
    
    question, context, answer = get_natural_questions_sample(data)
    if answer == '' and null_count < max_null:
        null_count += 1
        texts.append(context)
        dataset['question'].append(question)
        dataset['answer'].append(answer)
        continue
    elif answer != '':
        texts.append(context)
        dataset['question'].append(question)
        dataset['answer'].append(answer)

In [ ]:
chunks = []
i = 0
for text in tqdm(texts):
    chunks += semantic_chunker.split_text(text)
    i += 1

In [ ]:
base = QdrantBase(
    model_name='BAAI/bge-m3',
    vector_dimension=1024,
    collection_name='natural_questions_sentences',
    device='cuda',
)

In [ ]:
id_ = 0
for chunk in tqdm(chunks):
    base.add_point([chunk], {'id': id_})
    id_ += 1

# Оценка

In [ ]:
model = SimpleRag(collection_name='natural_questions_sentences')
df = pd.read_csv('research/data/qa_dataset.csv', index_col=0)

In [ ]:
natural_questions_dataset = {
    "question": [],
    "answer": [],
    "contexts": [],
    "ground_truth": [],
}

for i, (question, answer) in tqdm(df.loc[:200].iterrows(), total=df.loc[:200].shape[0]):
    result = model(question)
    natural_questions_dataset["question"].append(question)
    natural_questions_dataset["answer"].append(result['answer'])
    natural_questions_dataset["contexts"].append(result['context'])
    if pd.isna(answer):
        answer = 'No answer'
    natural_questions_dataset["ground_truth"].append(answer)    

In [ ]:
new_df = pd.DataFrame(natural_questions_dataset)
new_df.to_csv('research/data/semantic_chanling_rag.csv')

In [ ]:
from src.answer_correctness import AnswerCorrectness

evaluator = AnswerCorrectness()
res = evaluator.get_correctness(natural_questions_dataset['question'], natural_questions_dataset['answer'], natural_questions_dataset['ground_truth'])
np.mean(res[1])

In [ ]:
from src.ragas import Evaluator
ragas = Evaluator()

In [ ]:
e = ragas.eval(natural_questions_dataset)